<a href="https://colab.research.google.com/github/freakezoide/codigos-de-coolab/blob/main/modelo_con_regresion_lineal_y_dashboard.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
!pip install streamlit pyngrok
!npm install localtunnel

⠙⠹⠸⠼⠴⠦⠧
up to date, audited 23 packages in 1s
⠇
⠇3 packages are looking for funding
⠇  run `npm fund` for details
⠇
2 high severity vulnerabilities

To address all issues (including breaking changes), run:
  npm audit fix --force

Run `npm audit` for details.
⠇

In [10]:
%%writefile app.py
import streamlit as st
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
from sklearn.model_selection import train_test_split

# Configuración de la página
st.set_page_config(
    page_title="Dashboard Educativo",
    page_icon="📊",
    layout="wide",
    initial_sidebar_state="expanded"
)

# Título y descripción
st.title("📊 Dashboard de Análisis Educativo")
st.markdown("""
Análisis interactivo del uso de plataformas educativas por estudiantes y docentes.
""")

# Función para cargar datos
@st.cache_data
def load_data():
    try:
        # Datos de ejemplo ampliados para regresión
        estudiantes_data = {
            'Id': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
            'Sexo': ['F', 'M', 'F', 'M', 'F', 'M', 'F', 'M', 'F', 'M'],
            'Subsistema': ['DGES', 'DGETP', 'DGEIP', 'DGES', 'DGETP', 'DGEIP', 'DGES', 'DGETP', 'DGEIP', 'DGES'],
            'CREA': [5, 10, 8, 3, 7, 12, 6, 9, 4, 11],
            'Biblioteca': [2, 1, 3, 0, 2, 4, 1, 3, 0, 2],
            'PAM': [1, 2, 1, 0, 1, 3, 2, 1, 0, 2],
            'Horas_Estudio': [10, 15, 12, 8, 14, 18, 11, 16, 9, 17],
            'Departamento': ['Montevideo', 'Canelones', 'Maldonado', 'Salto', 'Paysandú',
                           'Rocha', 'Rivera', 'Artigas', 'Florida', 'Colonia']
        }
        estudiantes_df = pd.DataFrame(estudiantes_data)

        # Datos de ejemplo para docentes
        docentes_data = {
            'Id': [1, 2, 3, 4, 5],
            'Sexo': ['F', 'M', 'F', 'M', 'F'],
            'Subsistema': ['DGES', 'DGETP', 'DGEIP', 'DGES', 'DGETP'],
            'CREA': [8, 12, 9, 7, 11],
            'Biblioteca': [3, 2, 4, 1, 3],
            'PAM': [2, 3, 1, 2, 2],
            'Departamento': ['Montevideo', 'Canelones', 'Maldonado', 'Salto', 'Paysandú']
        }
        docentes_df = pd.DataFrame(docentes_data)

        # Calcular interactividad como variable objetivo para la regresión
        estudiantes_df['Interactividad'] = estudiantes_df[['CREA', 'Biblioteca', 'PAM']].sum(axis=1)
        docentes_df['Interactividad'] = docentes_df[['CREA', 'Biblioteca', 'PAM']].sum(axis=1)

        return estudiantes_df, docentes_df

    except Exception as e:
        st.error(f"Error cargando datos: {e}")
        return pd.DataFrame(), pd.DataFrame()

# Cargar datos
estudiantes_df, docentes_df = load_data()

# Sidebar con controles
st.sidebar.header("Filtros y Configuración")
rol = st.sidebar.radio("Seleccione el rol:", ["Estudiantes", "Docentes", "Comparación"], index=2)

analisis_type = st.sidebar.radio("Tipo de análisis:",
                                ["Visualización", "Modelo Predictivo - Clasificación",
                                 "Modelo Predictivo - Regresión"])

subsistemas = ["Todos"] + list(estudiantes_df['Subsistema'].unique())
subsistema_seleccionado = st.sidebar.selectbox("Subsistema:", subsistemas)

departamentos = ["Todos"] + list(estudiantes_df['Departamento'].unique())
departamento_seleccionado = st.sidebar.selectbox("Departamento:", departamentos)

# Función para aplicar filtros
def aplicar_filtros(df, subsistema, departamento):
    if subsistema != "Todos":
        df = df[df['Subsistema'] == subsistema]
    if departamento != "Todos":
        df = df[df['Departamento'] == departamento]
    return df

if analisis_type == "Modelo Predictivo - Regresión":
    st.header("📈 Modelo de Regresión Lineal")
    st.markdown("Predicción de la interactividad total basada en otros indicadores")

    # Cargar datos
    df = estudiantes_df if rol == "Estudiantes" else docentes_df if rol == "Docentes" else pd.concat([estudiantes_df, docentes_df])

    if not df.empty:
        # Selección de características
        st.subheader("Selección de Variables")
        default_features = ['CREA', 'Biblioteca', 'PAM', 'Horas_Estudio']
        available_features = [col for col in default_features if col in df.columns]

        features = st.multiselect(
            "Selecciona variables predictoras:",
            options=df.columns.drop(['Id', 'Interactividad', 'Sexo', 'Subsistema', 'Departamento']),
            default=available_features
        )

        target = 'Interactividad'

        if features and st.button("Entrenar Modelo de Regresión"):
            X = df[features]
            y = df[target]

            # Dividir datos
            X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

            # Entrenar modelo
            from sklearn.linear_model import LinearRegression
            from sklearn.metrics import mean_squared_error, r2_score

            model = LinearRegression()
            model.fit(X_train, y_train)

            # Evaluación
            y_pred = model.predict(X_test)
            mse = mean_squared_error(y_test, y_pred)
            r2 = r2_score(y_test, y_pred)

            # Mostrar resultados
            st.success("✅ Modelo entrenado exitosamente")

            col1, col2 = st.columns(2)
            col1.metric("Error Cuadrático Medio (MSE)", f"{mse:.2f}")
            col2.metric("Coeficiente R²", f"{r2:.2f}")

            # Coeficientes del modelo
            st.subheader("Coeficientes del Modelo")
            coef_df = pd.DataFrame({
                'Variable': ['Intercepto'] + features,
                'Coeficiente': [model.intercept_] + list(model.coef_)
            })
            st.dataframe(coef_df)

            # Gráfico de resultados
            st.subheader("Predicciones vs Valores Reales")
            fig, ax = plt.subplots()
            ax.scatter(y_test, y_pred)
            ax.plot([y.min(), y.max()], [y.min(), y.max()], 'k--', lw=2)
            ax.set_xlabel('Valores Reales')
            ax.set_ylabel('Predicciones')
            st.pyplot(fig)

            # Validación cruzada
            if st.checkbox("Mostrar validación cruzada"):
                from sklearn.model_selection import cross_val_score
                scores = cross_val_score(model, X, y, cv=5, scoring='r2')
                st.write("R² en validación cruzada:", scores)
                st.write("R² promedio:", scores.mean())

            # Gráfico de residuos
            if st.checkbox("Mostrar análisis de residuos"):
                residuals = y_test - y_pred
                fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 5))

                ax1.scatter(y_pred, residuals)
                ax1.axhline(y=0, color='r', linestyle='--')
                ax1.set_xlabel('Predicciones')
                ax1.set_ylabel('Residuos')
                ax1.set_title('Gráfico de Residuos')

                sns.histplot(residuals, kde=True, ax=ax2)
                ax2.set_title('Distribución de Residuos')

                st.pyplot(fig)

            # Predicción interactiva
            st.subheader("Realizar Predicción")
            input_data = {}
            cols = st.columns(len(features))
            for i, feature in enumerate(features):
                min_val = float(df[feature].min())
                max_val = float(df[feature].max())
                input_data[feature] = cols[i].slider(
                    f"Valor de {feature}",
                    min_value=min_val,
                    max_value=max_val,
                    value=float(df[feature].mean())
                )

            if st.button("Predecir Interactividad"):
                prediction = model.predict(pd.DataFrame([input_data]))
                st.success(f"Interactividad predicha: {prediction[0]:.2f}")
    else:
        st.warning("No hay datos disponibles para el análisis de regresión.")

# Visualizaciones principales
def mostrar_visualizaciones(est_df, doc_df, rol_seleccionado):
    if rol_seleccionado in ["Estudiantes", "Comparación"]:
        est_filtrado = aplicar_filtros(est_df, subsistema_seleccionado, departamento_seleccionado)
    if rol_seleccionado in ["Docentes", "Comparación"]:
        doc_filtrado = aplicar_filtros(doc_df, subsistema_seleccionado, departamento_seleccionado)

    if rol_seleccionado == "Comparación":
        st.header("Comparación entre Estudiantes y Docentes")

        col1, col2 = st.columns(2)

        with col1:
            st.subheader("Interactividad Total")
            fig = px.box(
                pd.DataFrame({
                    'Rol': ['Estudiantes']*len(est_filtrado) + ['Docentes']*len(doc_filtrado),
                    'Interactividad': list(est_filtrado['Interactividad']) + list(doc_filtrado['Interactividad'])
                }),
                x='Rol', y='Interactividad',
                title="Distribución de Interactividad Total"
            )
            st.plotly_chart(fig, use_container_width=True)

        with col2:
            st.subheader("Uso de Plataformas (Estudiantes)")
            plataformas_est = ['CREA', 'Biblioteca', 'PAM']

            promedios_est = est_filtrado.groupby('Subsistema')[plataformas_est].mean().reset_index()
            promedios_est_melt = promedios_est.melt(id_vars='Subsistema', var_name='Plataforma', value_name='Promedio')

            fig = px.bar(
                promedios_est_melt,
                x='Subsistema', y='Promedio',
                color='Plataforma',
                barmode='group'
            )
            st.plotly_chart(fig, use_container_width=True)

    elif rol_seleccionado == "Estudiantes":
        st.header("Análisis de Estudiantes")

        col1, col2 = st.columns(2)

        with col1:
            st.subheader("Distribución de Interactividad")
            fig = px.histogram(
                est_filtrado,
                x='Interactividad',
                nbins=20
            )
            st.plotly_chart(fig, use_container_width=True)

        with col2:
            st.subheader("Top Departamentos")
            top_deptos = est_filtrado.groupby('Departamento')['Interactividad'].mean().nlargest(5).reset_index()
            fig = px.bar(
                top_deptos,
                x='Interactividad',
                y='Departamento',
                orientation='h'
            )
            st.plotly_chart(fig, use_container_width=True)

    elif rol_seleccionado == "Docentes":
        st.header("Análisis de Docentes")

        col1, col2 = st.columns(2)

        with col1:
            st.subheader("Distribución de Interactividad")
            fig = px.histogram(
                doc_filtrado,
                x='Interactividad',
                nbins=20
            )
            st.plotly_chart(fig, use_container_width=True)

        with col2:
            st.subheader("Top Departamentos")
            top_deptos = doc_filtrado.groupby('Departamento')['Interactividad'].mean().nlargest(5).reset_index()
            fig = px.bar(
                top_deptos,
                x='Interactividad',
                y='Departamento',
                orientation='h'
            )
            st.plotly_chart(fig, use_container_width=True)

    # Mostrar datos filtrados
    st.subheader("Datos Filtrados")
    if rol_seleccionado == "Estudiantes":
        st.dataframe(est_filtrado)
    elif rol_seleccionado == "Docentes":
        st.dataframe(doc_filtrado)
    else:
        st.write("Estudiantes:")
        st.dataframe(est_filtrado)
        st.write("Docentes:")
        st.dataframe(doc_filtrado)

# Mostrar visualizaciones solo si no estamos en análisis de regresión
if analisis_type != "Modelo Predictivo - Regresión":
    mostrar_visualizaciones(estudiantes_df, docentes_df, rol)

# Información adicional
st.sidebar.markdown("---")
st.sidebar.markdown("""
**Instrucciones:**
1. Seleccione el rol a analizar
2. Filtre por subsistema y/o departamento
3. Explore las visualizaciones
""")

# Estadísticas generales
st.sidebar.markdown("---")
st.sidebar.subheader("Estadísticas Generales")
if not estudiantes_df.empty:
    st.sidebar.write(f"📚 Estudiantes: {len(estudiantes_df)}")
if not docentes_df.empty:
    st.sidebar.write(f"👩🏫 Docentes: {len(docentes_df)}")


Overwriting app.py


In [11]:
!streamlit run app.py &>/dev/null&
from pyngrok import ngrok

# Configura tu token de ngrok (obtenlo de https://dashboard.ngrok.com/get-started/your-authtoken)
ngrok.set_auth_token("2vjCFkars5ThIXUaSsyfGvD6XbO_2cxuD94UtUPw5dxdYwnwT")  # Reemplaza con tu token real

public_url = ngrok.connect(addr='8501', proto='http')
print("Tu aplicación está disponible en:", public_url)

Tu aplicación está disponible en: NgrokTunnel: "https://c22f-34-125-76-254.ngrok-free.app" -> "http://localhost:8501"
